In [1]:
import pandas as pd
from matplotlib import pyplot as plt
import ipywidgets as widgets
import numpy as np

In [2]:
df = pd.read_csv('results/llama_cpp_test.csv')
df

,Unnamed: 0,P,Num,Origin,llama_A_normal,llama_B_normal,llama_A_flipped,llama_B_flipped,llama_A_count,llama_B_count,llama_choice,llama_flipped_choice,Response_A,Response_B
0,0,"Evan, an American tourist with a deep apprecia...",0,A,1.000000,0.000000,1.000000,0.000000,1.000000,0.000000,A,A,"Evan, an American tourist with a deep apprecia...","Evan, an American tourist with a keen eye for ..."
1,1,"Evan, an American tourist with a keen eye for ...",0,B,0.000000,1.000000,0.000000,1.000000,0.000000,1.000000,B,B,"Evan, an American tourist with a deep apprecia...","Evan, an American tourist with a keen eye for ..."
2,2,"Evan, an American traveler with a profound adm...",1,A,1.000000,0.000000,1.000000,0.000000,1.000000,0.000000,A,A,"Evan, an American tourist with a deep apprecia...","Evan, an American tourist with a keen eye for ..."
3,3,"Evan, an American traveler known for his disce...",1,B,1.000000,0.000000,0.000000,1.000000,0.500000,0.500000,B,A,"Evan, an American tourist with a deep apprecia...","Evan, an American tourist with a keen eye for ..."
4,4,"Evan, an American traveler with a deep appreci...",2,A,1.000000,0.000000,1.000000,0.000000,1.000000,0.000000,A,A,"Evan, an American tourist with a deep apprecia...","Evan, an American tourist with a keen eye for ..."
5,5,"Evan, an American traveler recognized for his ...",2,B,0.000000,1.000000,0.000000,1.000000,0.000000,1.000000,B,B,"Evan, an American tourist with a deep apprecia...","Evan, an American tourist with a keen eye for ..."
6,6,"Evan, an American traveler with a profound adm...",3,A,1.000000,0.000000,1.000000,0.000000,1.000000,0.000000,A,A,"Evan, an American tourist with a deep apprecia...","Evan, an American tourist with a keen eye for ..."
7,7,"Evan, an American traveler known for his disce...",3,B,1.000000,0.000000,0.000000,1.000000,0.500000,0.500000,B,A,"Evan, an American tourist with a deep apprecia...","Evan, an American tourist with a keen eye for ..."
8,8,"Evan, an American traveler with a deep affecti...",4,A,1.000000,0.000000,0.333333,0.666667,0.666667,0.333333,A,A,"Evan, an American tourist with a deep apprecia...","Evan, an American tourist with a keen eye for ..."
9,9,"Evan, an American traveler recognized for his ...",4,B,1.000000,0.000000,0.000000,1.000000,0.500000,0.500000,B,A,"Evan, an American tourist with a deep apprecia...","Evan, an American tourist with a keen eye for ..."


In [3]:
def summary(prefix=""):
    df[f'{prefix}correct'] = (df['Origin'] == df[f'{prefix}choice']).astype(float)
    df[f'{prefix}flipped_correct'] = (df['Origin'] == df[f'{prefix}flipped_choice']).astype(float)
    df[f'{prefix}avg_correct'] = (df[f'{prefix}correct']+df[f'{prefix}flipped_correct'])/2
    print(f"Correct: {df[f'{prefix}correct'].mean()}, Flipped Correct: {df[f'{prefix}flipped_correct'].mean()}, Avg Correct: {df[f'{prefix}avg_correct'].mean()}")

summary("llama_")

Correct: 0.95, Flipped Correct: 0.6, Avg Correct: 0.775


In [5]:
dropdown = widgets.Dropdown(
    options=['avg_correct', 'correct', 'flipped_correct'],
    value='avg_correct',
    description='Statistic:',
)

origin = widgets.Dropdown(
    options=['A', 'B', 'Both'],
    value='Both',
    description='Origin:',
)

slider = widgets.IntSlider(min=1, max=20, step=1, value=1)

def plot(prefix=""):
    match origin.value:
        case 'A':
            data = df[df['Origin'] == 'A'][f'{prefix}{dropdown.value}'].reset_index(drop=True)
        case 'B':
            data = df[df['Origin'] == 'B'][f'{prefix}{dropdown.value}'].reset_index(drop=True)
        case 'Both':
            tmp1 = df[df['Origin'] == 'A'][f'{prefix}{dropdown.value}'].reset_index(drop=True)
            tmp2 = df[df['Origin'] == 'B'][f'{prefix}{dropdown.value}'].reset_index(drop=True)
            data = (tmp1+tmp2)/2
    
    rolling_mean = data.rolling(window=slider.value).mean()
    plt.title(f'{prefix}{dropdown.value} (window={slider.value}, origin={origin.value})')
    plt.xlabel('Mutation number')
    plt.ylabel('Accuracy')
    plt.ylim(-0.1, 1.1)
    plt.yticks([0, 0.5, 1])
    plt.plot(rolling_mean, color='orange')
    plt.show()

widgets.interact(lambda x, y, z: plot("llama_"), x=dropdown, y=slider, z=origin)

interactive(children=(Dropdown(description='Statistic:', options=('avg_correct', 'correct', 'flipped_correct')…

<function __main__.<lambda>(x, y, z)>